## Generating fixtures to test the Equation Of Time functions being used and the Longitude Calculation Functions

### Expected Longitude Output is downloaded directly from https://maps.nrel.gov/pvdaq/ PVDAQ Contributed Sites

### Expected longitude = -76.6636

In [1]:
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
# PVInsight Code Imports
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data
path = Path.cwd().parent.parent.parent
os.chdir(path)
# from pvsystemprofiler.estimator import ConfigurationEstimator
# from pvsystemprofiler.longitude_study import LongitudeStudy

### Load data table from external source

#### For today's example, we're loading data from NREL's PVDAQ API, which is a publically available PV generatation data set.

In [2]:
data_frame = get_pvdaq_data(sysid=1199, year=[2015, 2016, 2017], api_key='DEMO_KEY')

CAUTION: Multiple scan rates detected!=======================] 100.0% ...queries complete in 54.3 seconds       
Scan rates (in seconds): [300, 280]
0 transitions detected.
Suggest splitting data set between:




In [3]:
gmt_offset = -5

### Running the  DataHandler

In [4]:
#dh = DataHandler(data_frame)

In [5]:
dh = DataHandler(data_frame[0])

In [6]:
dh.run_pipeline(power_col='dc_power')

total time: 24.71 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              5.19s
Cleaning                   9.11s
Filtering/Summarizing      10.41s
    Data quality           0.44s
    Clear day detect       0.30s
    Clipping detect        8.28s
    Capacity change detect 1.38s



### Calculate the daily solar noon

In [7]:
from pvsystemprofiler.algorithms.optimized_sunrise_sunset import get_optimized_sunrise_sunset

In [8]:
fdm = dh.filled_data_matrix
rdm = None
day_of_year = dh.day_index.dayofyear.to_numpy()
days = dh.daily_flags.no_errors
opt_dict = get_optimized_sunrise_sunset(fdm, rdm)

In [9]:
sunrise = opt_dict['meas_sr_f']
sunset = opt_dict['meas_ss_f']
sunrise[np.isnan(sunrise)] = 0
sunset[np.isnan(sunset)] = 0
solarnoon = np.nanmean([sunrise, sunset], axis=0)

### Equation of time prosposed by Duffie And Rosa

In [10]:
path = Path.cwd()
os.chdir(path)
path

PosixPath('/Users/londonoh/Documents/github/pv-system-profiler')

In [11]:
# Exporting input for eot algorithms
np.savetxt('./tests/fixtures/longitude_calculation/eot_input.csv', day_of_year,  delimiter=",")

In [12]:
from pvsystemprofiler.utilities.equation_of_time import eot_da_rosa, eot_duffie

eot_duffie = eot_duffie(day_of_year)
eot_da_rosa = eot_da_rosa(day_of_year)

In [13]:
# Exporting output for eot algorithms
np.savetxt('./tests/fixtures/longitude_calculation/eot_duffie_output.csv', eot_duffie,  delimiter=",")
np.savetxt('./tests/fixtures/longitude_calculation/eot_da_rosa_output.csv', eot_da_rosa,  delimiter=",")

### Fit longitude

In [14]:
# exporting inputs for fit_longitude
# solarnoon
np.savetxt('./tests/fixtures/longitude_calculation/solarnoon.csv', solarnoon,  delimiter=",")
# days
np.savetxt('./tests/fixtures/longitude_calculation/days.csv', days,  delimiter=",")

In [17]:
from pvsystemprofiler.algorithms.longitude.calculation import calculate_longitude

calculate_longitude(eot_duffie, solarnoon, days, gmt_offset)

-91.40955141527715